In [ ]:
import torch
print("GPU Available:", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

# **DATA**

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, MultipleLocator, FuncFormatter
from matplotlib.lines import Line2D
from typing import Tuple, Optional, Union, Dict, List

np.random.seed(42)

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

torch.manual_seed(0)
from copy import deepcopy
from IPython.display import HTML

In [ ]:
class DNN(nn.Module):
    def __init__(self, input_dim, hidden_layers, hidden_units, activation, output_dim):
        super(DNN, self).__init__()

        layers = []
        in_features = input_dim

        for _ in range(hidden_layers):
            layers.append(nn.Linear(in_features, hidden_units))
            layers.append(activation())  # Activation function
            in_features = hidden_units

        # Output layer
        layers.append(nn.Linear(in_features, output_dim))
        
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


class CNN(nn.Module):
    def __init__(self, input_dim, num_filters, fc_units, dropout, num_conv_layers, output_dim):
        super(CNN, self).__init__()
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=1 if i == 0 else num_filters, out_channels=num_filters, kernel_size=3, padding=1)
            for i in range(num_conv_layers)
        ])
        self.fc = nn.Linear(num_filters * input_dim, fc_units)
        self.output = nn.Linear(fc_units, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        for conv in self.convs:
            x = torch.relu(conv(x))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(torch.relu(self.fc(x)))
        x = self.output(x)
        return x
        

class CNN_LSTM(nn.Module):
    def __init__(self, input_dim, num_filters, lstm_hidden, num_layers, fc_units, dropout, num_conv_layers, output_dim):
        super(CNN_LSTM, self).__init__()
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=1 if i == 0 else num_filters, out_channels=num_filters, kernel_size=3, padding=1)
            for i in range(num_conv_layers)
        ])
        self.lstm = nn.LSTM(input_size=num_filters, hidden_size=lstm_hidden, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(lstm_hidden, fc_units)
        self.output = nn.Linear(fc_units, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        for conv in self.convs:
            x = torch.relu(conv(x))
        x = x.permute(0, 2, 1)  # Reshape for LSTM (batch, seq_len, features)
        x, _ = self.lstm(x)
        x = self.dropout(torch.relu(self.fc(x[:, -1, :])))
        x = self.output(x)
        return x

In [ ]:
NUM_CLIENTS = 3  # Number of federated clients
input_dim = 25
NUM_ROUNDS = 10

# **MODELS**

In [ ]:
def DisplayTable(df: pd.DataFrame) -> None:
    """ Displays the data for each unique strategy in the DataFrame as an HTML table. """
    
    for st in df['Strategy'].unique().tolist():
        metrics = df[df['Strategy'] == st]
        display(HTML(metrics.to_html(index=False)))


def AggregateNodes(df: pd.DataFrame) -> pd.DataFrame:
    """ Average metrics across nodes. """

    results = []

    for (model, strategy), model_group in df.groupby(['Model', 'Strategy']):
        for rnd, round_group in model_group.groupby('Round'):
            epochs = round_group['Epochs'].iloc[0]

            nodes = sorted(round_group['Node'].unique())
            node_groups = [round_group[round_group['Node'] == n].reset_index(drop=True) for n in nodes]

            min_len = min(len(g) for g in node_groups)

            for i in range(min_len):
                global_step = rnd * epochs + i
                step_data = {
                    'Model': model,
                    'Strategy': strategy,
                    'GlobalRound': rnd,
                    'Round': global_step,
                    'TrainAcc': sum(g.loc[i, 'TrainAcc'] for g in node_groups) / len(node_groups),
                    'ValAcc': sum(g.loc[i, 'ValAcc'] for g in node_groups) / len(node_groups),
                    'TrainLoss': sum(g.loc[i, 'TrainLoss'] for g in node_groups) / len(node_groups),
                    'Epochs': epochs
                }
                results.append(step_data)

    return pd.DataFrame(results)

In [ ]:
model_palette = {
    'DNN': 'powderblue', 
    'CNN': 'navy',  
    'CNN-LSTM': 'royalblue'
}

def ModelComparisonTogether(df: pd.DataFrame, MODE: str, PARTITIONS: str, EPOCHS: int = 1) -> None:
    """ For each aggregation function, jointly plot Train and Validation accuracy of every model in each node. """

    handles_model = [Line2D([0], [0], color='w', markerfacecolor=model_palette.get(m, "grey"), marker='o', markersize=8, label=f"{m}") for m in df['Model'].unique()]
    handles_lstyle = [Line2D([0], [0], color='black', lw=2, linestyle='-', label='Train'),
                      Line2D([0], [0], color='black', lw=2, linestyle='--', label='Validation')]

    y_min_acc = min(min(df[df['Model'] == m]['TrainAcc'].min(), df[df['Model'] == m]['ValAcc'].min()) for m in df['Model'].unique())
    y_max_acc = max(max(df[df['Model'] == m]['TrainAcc'].max(), df[df['Model'] == m]['ValAcc'].max()) for m in df['Model'].unique()) + 0.001

    n_strategies = len(df['Strategy'].unique())
    
    for idx, st in enumerate(df['Strategy'].unique()):
        fig, axes = plt.subplots(nrows=1, ncols=NUM_CLIENTS, figsize=(15, 4))
        axes = axes.flatten()
        fig.suptitle(f"{MODE} - {PARTITIONS} - {st} - Mean Accuracy per round", fontsize=16, fontweight='black')

        x, y = (0.5, 0.88) if n_strategies < 5 else (0.5, 0.89)
        fig.text(x, y, f"Global rounds: {NUM_ROUNDS} - Local epochs: {EPOCHS}", ha='center', va='center', fontsize=12) 
    
        df_strategy = df[df['Strategy'] == st]

        for node in range(NUM_CLIENTS):
            ax_acc = axes[node]
            df_node = df_strategy[df_strategy['Node'] == node]
            
            for model in df['Model'].unique().tolist():
                model_str_df = df_node[df_node['Model'] == model]
                color = model_palette.get(model, "grey")
                    
                ax_acc.plot(model_str_df['Round'] + 1, model_str_df['TrainAcc'], marker='o', markersize=3, linestyle='-', linewidth=1, color=color, label=f"{model} - Train")
                ax_acc.plot(model_str_df['Round'] + 1, model_str_df['ValAcc'], marker='s', markersize=3, linestyle='dashed', linewidth=1, color=color, label=f"{model} - Val")
                        
                ax_acc.set_xlabel("Round")
                            
            ax_acc.set_title(f"Node {node+1}")
            ax_acc.set_ylim(y_min_acc, y_max_acc)
            ax_acc.xaxis.set_major_locator(MultipleLocator(2))
            ax_acc.xaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x)}'))
            ax_acc.grid(True, alpha=0.7)
    
            if idx == 0 or idx == 0:
                ax_acc.set_ylabel('Accuracy')
                    
        if len(axes) > n_strategies:  
            axes[-1].axis('off')

        section_title_style = {'color': 'black', 'lw': 0}
        combined_legend = [
            Line2D([], [], **section_title_style, label=r"$\\mathbf{Accuracy\\ Type}$"),
            *handles_lstyle,
            Line2D([], [], **section_title_style, label=r"$º\mathbf{Model}$"),
            *handles_model
        ]
        axes[-1].legend(handles=combined_legend, fontsize=8)
        
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.show()
        plt.close(fig)


def ModelComparisonAvgTogetherPart(df: pd.DataFrame, MODE: str, EPOCHS: int = 1) -> None:
    """ For each aggregation function, jointly plot the averaged Train and Validation accuracy of each model. """

    handles_model = [Line2D([0], [0], color='w', markerfacecolor=model_palette.get(m, "grey"), marker='o', markersize=8, label=f"{m}") for m in df['Model'].unique()]
    handles_lstyle = [Line2D([0], [0], color='black', lw=2, linestyle='-', label='Train'),
                      Line2D([0], [0], color='black', lw=2, linestyle='--', label='Validation')]

    y_min_acc = min(min(df[df['Model'] == m]['TrainAcc'].min(), df[df['Model'] == m]['ValAcc'].min()) for m in df['Model'].unique())
    y_max_acc = max(max(df[df['Model'] == m]['TrainAcc'].max(), df[df['Model'] == m]['ValAcc'].max()) for m in df['Model'].unique()) + 0.001

    for partition in df['Partition'].unique():
        df_part = df[df['Partition'] == partition]
        
        n_strategies = len(df['Strategy'].unique())
        nrows = 2 if n_strategies > 4 else 1
        ncols = (n_strategies + 1) // 2 if n_strategies > 4 else n_strategies 
    
        height = nrows * 0.5 if nrows == 2 else nrows * 5
        width = ncols * 5 if n_strategies > 4 else ncols * 6 
        
        fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(width, height))
        if n_strategies == 1:
            axes = [axes] 
        else:
            axes = axes.flatten() if nrows > 1 else list(axes)
    
        fig.suptitle(f"{MODE} - {partition} - Mean Accuracy per round", fontsize=16, fontweight='black')
    
        x, y = (0.5, 0.9) if n_strategies < 5 else (0.5, 0.93)
        fig.text(x, y, f"Global rounds: {NUM_ROUNDS} - Local epochs: {EPOCHS}", ha='center', va='center', fontsize=12) 
        
        for idx, st in enumerate(df['Strategy'].unique()):
            df_strategy = df_part[df_part['Strategy'] == st]
            
            for model in df['Model'].unique().tolist():
                model_str_df = df_strategy[df_strategy['Model'] == model]
                color = model_palette.get(model, "grey")
            
                ax_acc = axes[idx]
                    
                ax_acc.plot(model_str_df['Round'] + 1, model_str_df['TrainAcc'], marker='o', markersize=3, linestyle='-', color=color, label=f"{model} - Train")
                ax_acc.plot(model_str_df['Round'] + 1, model_str_df['ValAcc'], marker='s', markersize=3, linestyle='dashed', color=color, label=f"{model} - Val")
                        
                ax_acc.set_xlabel("Round")                            
                ax_acc.set_title(f"{st}")
                ax_acc.set_ylim(y_min_acc, y_max_acc)
                if NUM_ROUNDS > 10:
                    ax_acc.xaxis.set_major_locator(MultipleLocator(2))
                else:
                    ax_acc.xaxis.set_major_locator(MaxNLocator(integer=True))
                ax_acc.xaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x)}'))
                ax_acc.grid(True, alpha=0.7)
        
                if idx == 0 or idx == ncols:
                    ax_acc.set_ylabel('Accuracy')
                    
        if len(axes) > n_strategies:  
            axes[-1].axis('off')

        section_title_style = {'color': 'black', 'lw': 0}
        combined_legend = [
            Line2D([], [], **section_title_style, label=r"$\\mathbf{Accuracy\\ Type}$"),
            *handles_lstyle,
            Line2D([], [], **section_title_style, label='\n'),
            Line2D([], [], **section_title_style, label=r"$\\mathbf{Model}$"),
            *handles_model
        ]
    
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.show()

In [ ]:
agg_palette = {
    'UNIFORM': 'gray', 
    'UNBALANCED': 'firebrick'
}

def SingleModelAnalysis(df: pd.DataFrame, MODE: str) -> None:
    """ Compare partitions. For each node, plot Train and Validation accuracy for both partitions in different training configurations. """

    handles_epoch = [Line2D([0], [0], color='w', markerfacecolor=agg_palette.get(part, "grey"), marker='o', markersize=8, label=f"{part}") for part in df['Partition'].unique()]
    handles_lstyle = [Line2D([0], [0], color='black', lw=2, linestyle='-', label='Train'),
                      Line2D([0], [0], color='black', lw=2, linestyle='--', label='Validation')]

    section_title_style = {'color': 'black', 'lw': 0}
    combined_legend = [
        Line2D([], [], **section_title_style, label=r"$\\mathbf{Accuracy\\ Type}$"),
        *handles_lstyle,
        Line2D([], [], **section_title_style, label=r"$\\mathbf{Data\\ Partition}$"),
        *handles_epoch
    ]
    
    if (MODE == 'AC') and ('BalValAcc' in df.columns):
        handles_lstyle.append(Line2D([0], [0], color='black', lw=2, linestyle=':', label='Balanced Val'))

    for strategy in df['Strategy'].unique():
        df_metrics = df[df['Strategy'] == strategy]
        y_min_acc = min(df_metrics[['TrainAcc', 'ValAcc']].min())
        y_max_acc = max(df_metrics[['TrainAcc', 'ValAcc']].max())
        
        for epoch_val in df_metrics['Epochs'].unique():
            epochs_df = df_metrics[df_metrics['Epochs'] == epoch_val]
    
            fig, axes = plt.subplots(nrows=1, ncols=NUM_CLIENTS, figsize=(15, 3.5))
        
            fig.subplots_adjust(top=0.78, bottom=0.15, hspace=0.9)
            if epoch_val == 1:
                fig.suptitle(f"{MODE} - {df_metrics['Model'].unique()[0]} - {df_metrics['Strategy'].unique()[0]}", fontsize=16, fontweight='bold')
            fig.text(0.5, 0.89, f"Global rounds: {NUM_ROUNDS} - Local epochs: {epoch_val}", ha='center', va='center', fontsize=12)
        
            for node in range(NUM_CLIENTS):
                ax_acc = axes[node]
                
                for part in df_metrics['Partition'].unique():
                    part_str_df = epochs_df[epochs_df['Partition'] == part]
                    node_data = part_str_df[part_str_df['Node'] == node]
                    x_epochs = np.arange(1, len(node_data) + 1) 
                    col = agg_palette.get(part, "grey")
                    
                    ax_acc.plot(x_epochs, node_data['TrainAcc'], marker='o', markersize=0, linestyle='-', linewidth=1.5, color=col, label=f"{part} (Train)")
                    ax_acc.plot(x_epochs, node_data['ValAcc'], marker='s', markersize=0, linestyle='--', linewidth=1, color=col, label=f"{part} (Val)")
                    if MODE == 'AC' and part == 'UNBALANCED' and epoch_val != 1 and ('BalValAcc' in df.columns):
                        ax_acc.plot(x_epochs, node_data['BalValAcc'], marker='s', markersize=0, linestyle=':', linewidth=1, color=col, label=f"{part} (Bal Val)")

                tick_positions = []
                i = 0
                while (tick := epoch_val * i) <= max(x_epochs):
                    if tick >= min(x_epochs):
                        tick_positions.append(tick)
                    i += 1
                if tick_positions[-1] != max(x_epochs):
                    tick_positions.append(max(x_epochs))

                if MODE == 'AC' and NUM_ROUNDS == 20:
                    labels = [f"{tick}" if idx % 2 == 0 else "" for idx, tick in enumerate(tick_positions)]
                else:
                    labels = [f"{tick}" for tick in tick_positions]
                
                ax_acc.set_xticks(tick_positions, [f"{tick}" for tick in tick_positions])
                ax_acc.set_xlabel("Epoch")
                ax_acc.set_xticklabels(labels)
                ax_acc.set_title(f"Node {node+1}")
                ax_acc.set_ylim(y_min_acc, y_max_acc)
                ax_acc.grid(True, alpha=0.7)
    
                if node == 0:
                    ax_acc.set_ylabel("Accuracy")
                if node == 2:
                    ax_acc.legend(handles=combined_legend, fontsize=7)
    
            plt.show()

## **ATTACK DETECTION**

**STAGE 1**. Model and Aggregation function

In [ ]:
MODE = 'AD'
EPOCHS = 1
NUM_ROUNDS = 10

df_AD_metrics, df_AD_metrics_avg = {}, pd.DataFrame()
cond = (df_AD_metrics_avg['Strategy'] != 'FedProx (1)') & (df_AD_metrics_avg['Strategy'] != 'FedProx (0.1)')

for PARTITIONS in ['UNIFORM', 'UNBALANCED']:

    print(f" \n\n --------------------------- {PARTITIONS} ---------------------------")

    df_AD_metrics[f"{PARTITIONS}{EPOCHS}"] = pd.read_csv(f"..\\metrics\\{MODE}\\{MODE}{EPOCHS}_{PARTITIONS}_Metrics.csv")
    df_AD_metrics[f"{PARTITIONS}{EPOCHS}"]['Epochs'] = EPOCHS
    
    ModelComparisonTogether(df_AD_metrics[f"{PARTITIONS}{EPOCHS}"], MODE, PARTITIONS)
    ModelComparisonTogether(df_AD_metrics[f"{PARTITIONS}{EPOCHS}"][cond], MODE, PARTITIONS)
    
    # Aggregate metrics and analyze
    df_AD_avg = AggregateNodes(df_AD_metrics[f"{PARTITIONS}{EPOCHS}"], MODE, PARTITIONS)
    df_AD_avg['Partition'] = PARTITIONS
    df_AD_metrics_avg = pd.concat([df_AD_metrics_avg, df_AD_avg], ignore_index=True)

print(f" \n\n --------------------------- BOTH PARTITIONS ---------------------------")
# ModelComparisonAvgTogetherPart(df_AD_metrics_avg, MODE, EPOCHS)
ModelComparisonAvgTogetherPart(df_AD_metrics_avg[cond], MODE, EPOCHS)

**STAGE 2**. Training configuration

In [ ]:
df_AD_metrics_CNN = pd.DataFrame()

for PARTITIONS in ['UNIFORM', 'UNBALANCED']:
    for EPOCHS in [1, 3, 5, 10]:
    
        df_full = pd.read_csv(f"..\\metrics\\{MODE}\\{MODE}{EPOCHS}_{PARTITIONS}_Metrics.csv")
        temp_df = df_full[(df_full['Model'] == 'CNN') & ((df_full['Strategy'] == 'FedAvg') | (df_full['Strategy'] == 'GeomMedian') )].copy()
        temp_df['Epochs'] = EPOCHS
        temp_df['Partition'] = PARTITIONS
        df_AD_metrics_CNN = pd.concat([df_AD_metrics_CNN, temp_df], ignore_index=True)

SingleModelAnalysis(df_AD_metrics_CNN, MODE)

## **ATTACK CLASSIFICATION**

**STAGE 1**. Model and Aggregation function

In [ ]:
MODE = 'AC'
EPOCHS = 1
NUM_ROUNDS = 10

df_AC_metrics, df_AC_metrics_avg = {}, pd.DataFrame()
for PARTITIONS in ['UNIFORM', 'UNBALANCED']:

    print(f" \n\n --------------------------- {PARTITIONS} ---------------------------")

    df_AC_metrics[f"{PARTITIONS}{EPOCHS}"] = pd.read_csv(f"..\\metrics\\{MODE}\\{MODE}{EPOCHS}_{PARTITIONS}_Metrics.csv")
    df_AC_metrics[f"{PARTITIONS}{EPOCHS}"] = df_AC_metrics[f"{PARTITIONS}{EPOCHS}"][df_AC_metrics[f"{PARTITIONS}{EPOCHS}"]['Round'] < NUM_ROUNDS]
    df_AC_metrics[f"{PARTITIONS}{EPOCHS}"]['Epochs'] = EPOCHS
    
    ModelComparisonTogether(df_AC_metrics[f"{PARTITIONS}{EPOCHS}"], MODE, PARTITIONS)
    
    # Aggregate metrics and analyze
    df_AC_avg = AggregateNodes(df_AC_metrics[f"{PARTITIONS}{EPOCHS}"], MODE, PARTITIONS)
    df_AC_avg['Partition'] = PARTITIONS
    df_AC_metrics_avg = pd.concat([df_AC_metrics_avg, df_AC_avg], ignore_index=True)

print(f" \n\n --------------------------- BOTH PARTITIONS ---------------------------")
ModelComparisonAvgTogetherPart(df_AC_metrics_avg, MODE, EPOCHS)

**STAGE 2**. Training configuration

In [ ]:
df_AC_metrics_CNN = pd.DataFrame()
NUM_ROUNDS = 20

for PARTITIONS in ['UNIFORM', 'UNBALANCED']:
    for EPOCHS in [1, 3, 5, 10]:
        
        df_full = pd.read_csv(f"..\\metrics\\{MODE}\\{MODE}{EPOCHS}_{PARTITIONS}_Metrics.csv")
        temp_df = df_full[(df_full['Model'] == 'CNN') & (df_full['Strategy'] == 'GeomMedian') ].copy()
        temp_df['Epochs'] = EPOCHS
        temp_df['Partition'] = PARTITIONS
        df_AC_metrics_CNN = pd.concat([df_AC_metrics_CNN, temp_df], ignore_index=True)

condition  = (df_AC_metrics_CNN['Partition'] == 'UNBALANCED') & (df_AC_metrics_CNN['Epochs'] != 1)
df_AC_metrics_CNN.loc[condition, 'ValAcc'] = df_AC_metrics_CNN.loc[condition, 'BalValAcc']
df_AC_metrics_CNN = df_AC_metrics_CNN.drop(columns=['BalValAcc'])

SingleModelAnalysis(df_AC_metrics_CNN, MODE)

# **DIFFERENTIAL PRIVACY**

In [ ]:
def DPAnalysis(df: pd.DataFrame, MODE: str, val = '') -> None:
    """ For each partition and node, plot Train and Validation Accuracy comparing different values of "val". """

    y_min_acc = min(df[['TrainAcc', 'ValAcc']].min())
    y_max_acc = max(df[['TrainAcc', 'ValAcc']].max())

    cmap = plt.get_cmap('tab20c')
    
    for part in df['Partition'].unique():
        df_part = df[df['Partition'] == part]

        # Style legend for accuracy types
        handles_lstyle = [
            Line2D([0], [0], color='black', lw=2, linestyle='-', label='Train'),
            Line2D([0], [0], color='black', lw=2, linestyle='--', label='Validation')
        ]

        # Color legend for training modes
        mode_parts = list(df_part['Mode'].unique())

        indices = []
        i = 4
        while i < 20:
            indices.extend([i, i+1, i+2])
            i += 4 
        color_indices = [idx for idx in indices if idx < 20]
        
        if 'Epsilon' in val:
            cmap = plt.get_cmap('tab20b')
            color_map = { mode: cmap(color_indices[i % len(color_indices)]) for i, mode in enumerate(mode_parts[1:])}
       
        elif 'Max gradient norm' in val:
            color_map = { mode: cmap(color_indices[i % len(color_indices)]) for i, mode in enumerate(mode_parts[1:])}
            
        else:
            predefined_indices = [6, 14, 8, 15] if MODE == 'AD' else [8, 14, 6, 15]
            color_map = { mode: cmap(predefined_indices[i % len(predefined_indices)]) for i, mode in enumerate(mode_parts[1:])}
        
        handles_modes = [
            Line2D([0], [0], color='w', markerfacecolor=color_map.get(mp, "grey"), marker='o', markersize=8, label=mp.split('_')[0])
            for mp in mode_parts
        ]

        if MODE == 'AD':
            fig, axes = plt.subplots(nrows=1, ncols=NUM_CLIENTS, figsize=(18, 4))
        else:
            fig, axes = plt.subplots(nrows=1, ncols=NUM_CLIENTS, figsize=(20, 5))
        fig.subplots_adjust(top=0.8, bottom=0.15, hspace=0.7)
        fig.suptitle(f"{MODE} - {part} PARTITIONS - ACCURACY COMPARISON USING DIFFERENTIAL PRIVACY", fontsize=16, fontweight='bold')
        fig.text(0.5, 0.9, f"Architecture: {df.iloc[0]['Model']} - Aggregation function: {df.iloc[0]['Strategy']} - Global rounds: {NUM_ROUNDS} - Local epochs: {EPOCHS} {val}", ha='center', va='center', fontsize=12)
        
        for node in range(NUM_CLIENTS):
            ax_acc = axes[node]

            for i, mod in enumerate(df_part['Mode'].unique()):
                part_mod_df = df_part[df_part['Mode'] == mod]
                node_data = part_mod_df[part_mod_df['Node'] == node]
                x_epochs = np.arange(1, len(node_data) + 1) 
                
                mod_part = node_data['Mode'].iloc[0] # mod + "_" + part
                col = color_map.get(mod_part, "grey")

                ax_acc.plot(x_epochs, node_data['TrainAcc'], marker='o', markersize=0, linestyle='-', linewidth=1, color=col)
                ax_acc.plot(x_epochs, node_data['ValAcc'], marker='s', markersize=0, linestyle='--', linewidth=1, color=col)
                
                # Tick formatting
                tick_positions = []
                i = 0
                while (tick := EPOCHS * i) <= max(x_epochs):
                    if tick >= min(x_epochs):
                        tick_positions.append(tick)
                    i += 1
                if tick_positions[-1] != max(x_epochs):
                    tick_positions.append(max(x_epochs))
                
                labels = [f"{tick}" for tick in tick_positions]
                
                ax_acc.set_xticks(tick_positions)
                ax_acc.set_xticklabels(labels)
                ax_acc.set_xlabel("Epoch")
                ax_acc.set_title(f"Node {node+1}")
                ax_acc.set_ylim(y_min_acc, y_max_acc)
                ax_acc.grid(True, alpha=0.7)

                if node == 0:
                    ax_acc.set_ylabel("Accuracy")

            section_title_style = {'color': 'black', 'lw': 0}
            pseudo_handles = [
                Line2D([], [], **section_title_style, label=r"$\\mathbf{Accuracy\\ Type}$"),
                *handles_lstyle,
                Line2D([], [], **section_title_style, label=' '),
                Line2D([], [], **section_title_style, label=r"$\\mathbf{Training\\ Mode}$"),
                *handles_modes
            ]

            if node == 2: 
                ax_acc.legend(handles=pseudo_handles, fontsize=6, title_fontsize=8)
        plt.show()

In [ ]:
def ExtractData(MODE:str, df: pd.DataFrame, feature: str, max_grad_norms: List, epsilons: List, all_features: List = []) -> pd.DataFrame:
    """ Extracts and aggregates experimental DP results from stored CSV metric files based on different training configurations. """

    NUM_ROUNDS = 10
    EPOCHS = 5
    
    for PARTITIONS in ['UNIFORM', 'UNBALANCED']:
        temp_df_single = pd.read_csv(f"..\\metrics\\{MODE}\\{MODE}{EPOCHS}_{PARTITIONS}_Metrics.csv")
        temp_df_single = temp_df_single[(temp_df_single['Strategy'] == 'GeomMedian') & (temp_df_single['Round'] < NUM_ROUNDS)].copy()
        temp_df_single['Epochs'] = EPOCHS
        temp_df_single['Partition'] = PARTITIONS
        temp_df_single['Mode'] = f"{feature} = inf" if feature else "eps = inf, thr = inf"
        temp_df_single['Epsilon'] = np.inf
        temp_df_single['MaxGradNorm'] = np.inf
        df = pd.concat([df, temp_df_single], ignore_index=True)
    
        for eps in epsilons:
            for max_grad_norm in max_grad_norms:
                file_path = f"..\\metrics\\{MODE}\\DP\\{MODE}{EPOCHS}-DP{eps}{max_grad_norm}_{PARTITIONS}_Metrics.csv"
                if os.path.exists(file_path):
                    temp_df = pd.read_csv(file_path)

                    max_grad_norm = 1 if max_grad_norm == '' else max_grad_norm.replace('-', '')
                    temp_df['MaxGradNorm'] = float(max_grad_norm)
                    temp_df['Epochs'] = EPOCHS
                    temp_df['Partition'] = PARTITIONS
                    
                    if feature:
                        val = eps if feature == 'eps' else max_grad_norm
                        temp_df['Mode'] = f"{feature} = {val}"
                    else:
                        grad_norm = all_features[f"{MODE}{eps}"]
                        temp_df['Mode'] = f"eps = {str(eps).split('-')[0]}, thr = {grad_norm}"
                    df = pd.concat([df, temp_df], ignore_index=True)

    if MODE == 'AC':
        condition  = (df['Partition'] == 'UNBALANCED')
        df.loc[condition, 'ValAcc'] = df.loc[condition, 'BalValAcc']
        df = df.drop(columns=['BalValAcc'])
            
    return df

### **ATTACK DETECTION**

Compare different privacy budgets $\epsilon$ values

In [ ]:
MODE = 'AD'
df_ADDP_metrics = pd.DataFrame()

df_ADDP_metrics = ExtractData(MODE, df_ADDP_metrics, 'eps', ['-1'], [0.5, 1, 2, 3, 5, 10, 20])
max_grad_norm = 1
DPAnalysis(df_ADDP_metrics, MODE, f' - Max gradient norm: {max_grad_norm}')

Compare different Max clipping threshold $\mathcal{C}$ values

In [ ]:
df_ADDP_metrics = pd.DataFrame()

max_grad_norms = ['-0.5', '-0.75', '', '-1.5', '-2']
epsilons = [2]
df_ADDP_metrics = ExtractData(MODE, df_ADDP_metrics, 'thr', max_grad_norms, epsilons)
DPAnalysis(df_ADDP_metrics, MODE, f' - Epsilon: {epsilons[0]}')

Comparison of the best $\epsilon$ and $\mathcal{C}$ already found and the ones achieved doing hyperparameter tunning

In [ ]:
df_ADDP_metrics = pd.DataFrame()

AD_DP_HT = {'AD2': 1, 'AD2.52-1.97': 1.97} # key: MODE + epsilon, value: max clipping norm
max_grad_norms = ['']
epsilons = [2, '2.52-1.97']
df_ADDP_metrics = ExtractData(MODE, df_ADDP_metrics, '', max_grad_norms, epsilons, AD_DP_HT)
DPAnalysis(df_ADDP_metrics, MODE)

## **ATTACK CLASSIFICATION**

Compare different privacy budgets $\epsilon$ values

In [ ]:
df_ACDP_metrics = pd.DataFrame()

epsilons = [0.5, 1, 2, 3, 5, 10, 20]
df_ACDP_metrics = ExtractData(MODE, df_ACDP_metrics, 'eps', ['-1'], epsilons)
max_grad_norm = 1
DPAnalysis(df_ACDP_metrics, MODE, f' - Max gradient norm: {max_grad_norm}')

Compare different Max clipping threshold $\mathcal{C}$ values

In [ ]:
MODE = 'AC'
df_ACDP_metrics = pd.DataFrame()

max_grad_norms = ['-0.5', '-0.75', '-1', '-1.5', '-2']
epsilons = [3]
df_ACDP_metrics = ExtractData(MODE, df_ACDP_metrics, 'thr', max_grad_norms, epsilons)
DPAnalysis(df_ACDP_metrics, MODE, f' - Epsilon: {epsilons[0]}')

Comparison of the best $\epsilon$ and $\mathcal{C}$ already found and the ones achieved doing hyperparameter tunning

In [ ]:
MODE = 'AC'
df_ACDP_metrics = pd.DataFrame()

AC_DP_HT = {'AC3-1': 1, 'AC2.13-1.99': 1.99} # key: MODE + epsilon, value: max clipping norm
max_grad_norms = ['']
epsilons = ['3-1', '2.13-1.99']
df_ACDP_metrics = ExtractData(MODE, df_ACDP_metrics, '', max_grad_norms, epsilons, AC_DP_HT)
DPAnalysis(df_ACDP_metrics, MODE)